If not yet available some libraries and their python bindings have to be installed :<br>
- gmsh (best installed globally through package management system)
- python3 -m pip install pygmsh --user
- VTK (best installed globally through package management system)
- python3 -m pip install vtk --user

In [1]:
import numpy as np
from scipy import constants
import pygmsh
from MeshedFields import *

# Create a meshed screen with a central hole

Typical cell size is 2 mm along the outside of the screen and 0.5 mm near the inner hole.<br>
The wavelength is 0.3 mm only. That means we need a 0.02 mm resolution along the x/z axis.<br>
When generating the geometry we stretch it by a factor 20 and undo thatafter creating the mesh,
effectively generating cells that are denser along the the stretched direction.

In [2]:
with pygmsh.geo.Geometry() as geom:
    
    Ra = 0.050
    Ri = 0.002
    lca = 0.002
    lci = 0.0005
    stretch = 20.0

    # we create the initial geometry as a streched ellipse to create
    # different scaling lengths (cell sizes) along the different axes
    p1 = geom.add_point([Ra*stretch, 0.0], lca)
    p2 = geom.add_point([0.0, Ra], lca)
    p3 = geom.add_point([-Ra*stretch, 0.0], lca)
    p4 = geom.add_point([0.0, -Ra], lca)
    p1i = geom.add_point([Ri*stretch, 0.0], lci)
    p2i = geom.add_point([0.0, Ri], lci)
    p3i = geom.add_point([-Ri*stretch, 0.0], lci)
    p4i = geom.add_point([0.0, -Ri], lci)
    pc = geom.add_point([0.0, 0.0])
    pa = geom.add_point([0.0, 1.0])

    # the mesh is circumscribed with four elliptic arcs
    e1 = geom.add_ellipse_arc(start=p1, center=pc, point_on_major_axis=pa, end=p2)
    e2 = geom.add_ellipse_arc(start=p2, center=pc, point_on_major_axis=pa, end=p3)
    e3 = geom.add_ellipse_arc(start=p3, center=pc, point_on_major_axis=pa, end=p4)
    e4 = geom.add_ellipse_arc(start=p4, center=pc, point_on_major_axis=pa, end=p1)
    outline = geom.add_curve_loop([e1, e2, e3, e4])

    # the hole is circumscribed with four elliptic arcs
    e1i = geom.add_ellipse_arc(start=p1i, center=pc, point_on_major_axis=pa, end=p2i)
    e2i = geom.add_ellipse_arc(start=p2i, center=pc, point_on_major_axis=pa, end=p3i)
    e3i = geom.add_ellipse_arc(start=p3i, center=pc, point_on_major_axis=pa, end=p4i)
    e4i = geom.add_ellipse_arc(start=p4i, center=pc, point_on_major_axis=pa, end=p1i)
    hole = geom.add_curve_loop([e1i,e2i,e3i,e4i])

    pl = geom.add_plane_surface(outline, holes=[hole])
    
    mesh = geom.generate_mesh()

In [3]:
# un-stretch
pts = np.array([np.array([p[0]/stretch,p[1],0.0]) for p in mesh.points])
tris = mesh.cells_dict['triangle']
screen = MeshedField(pts,tris)

The screen is placed at z=3.625 m from th origin. A beam is assumed to propagate in z direction<br>
The fields shall be reflected to th x direction. The screen normal is pointing in the negative z and positive x direction (x - left, y - up).

In [4]:
pts = np.array([np.array([p[0],p[1],p[0]])+np.array([0.0,0.0,3.625]) for p in pts])
screen = MeshedField(pts,tris)

In [5]:
mesh

<meshio mesh object>
  Number of points: 57151
  Number of cells:
    line: 2336
    triangle: 111962
    vertex: 10

In [11]:
print("%d points" % len(screen.points))
print("%d triangles" % len(screen.triangles))
area = screen.MeshArea()
normals = screen.MeshNormals()
average = np.sum(normals, axis=0)/screen.Np
print("total mesh area = %7.3f cm²" % (1.0e4*np.sum(area)))
print("screen normal = %s" % average)
screen.ShowMeshedField(showAxes=True)

57151 points
111962 triangles
total mesh area = 110.893 cm²
screen normal = [ 7.07106781e-01  1.67503047e-18 -7.07106781e-01]


### define the timing
The beam is assumed to start at t=0. The fields are propagating with c so the expected time a
signal arrives at some screen point is z/c.

In [8]:
# time step
screen.dt = 0.5e-13
# some time shift of the waveform start
delay = 65.0e-12
# all points use the same timing grid
screen.Nt = 600
screen.t0 = np.array([p[2]/constants.c-screen.Nt/2*screen.dt for p in screen.pos])

### Write the file
It is assumed that the script was started from the scripts directory. The file should be created in the tests folder.

In [9]:
filename="MirrorWithHole.h5"
screen.WriteMeshedField(filename)

not writing ElMagField dataset
